In [ ]:
import cv2 as cv 
import matplotlib.pyplot as plt
import numpy

In [ ]:
Obter esqueleto por imagem

In [ ]:
net = cv.dnn.readNetFromTensorflow("graph_opt.pb")  ## weights

In [5]:
inWidth  = 368
inHeight = 368
thr      = 0.2

In [ ]:
## define partes do corpo e linhas entre eles. Os pontos do corpo são identificadas por machine learning,
## e eles são ligados pelas linhas, os pares da pose
    BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
                   "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
                   "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
                   "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

    POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
                   ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
                   ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
                   ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
                   ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

In [ ]:
img = cv.imread("image.jpg")  ## atribui a imagem à variável

In [ ]:
plt.imshow(img)               ## mostra a imagem, sem nenhum esqueleto. É um teste para ver se funciona

In [ ]:
Obter por uma webcam

In [ ]:

## Obtém captura de tela
cap = cv.VideoCapture(1) 
cap.set(cv.CAP_PROP_FPS,10)
cap.set = (3,800)
cap.set = (4,800)

## Atua até uma tecla ser pressionada
if cap.isOpened:   ## canal aberto? captura
    cap = cv.VideoCapture(0) 
if not cap.isOpened: ## caso contrario, exibe mensagem de erro
    raise IOError("Nao ta abrindo webcam")  
    
while cv.waitKey(1) < 0
    hasFrame, frame = cap.read()    ## se tem o frame, le o quadro
    if not hasFrame
        cv.waitKey()                ## caso contrario, espera uma tela
        break
        
## Prepara largura e altura baseadas na forma obtida pelo frame capturado         
    frameWidth = frame.shape[1]  ## largura do frame
    frameHeigth = frame.shape[0] ## altura  do frame
    inp = cv.dnn.blobFromImage(frame, inScale, (inWidth, inHeight),(0, 0, 0), swapRB=False, crop=False)
    net.setInput(inp)
    out = net.forward()

    assert(len(BODY_PARTS) <= out.shape[1])

    points = []   ## Lista de pontos, o nosso "esqueleto"
    for i in range(len(BODY_PARTS)):
        # separa um mapa de calor da peça i do corpo na foto 
        heatMap = out[0, i, :, :]

        # Originally, we try to find all the local maximums. To simplify a sample
        # we just find a global one. However only a single pose at the same time
        # could be detected this way.
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth  * point[0]) / out.shape[3]   ## obtem um x normalizado para o video
        y = (frameHeight * point[1]) / out.shape[2]   ## obtem um y normalizado para o video

        # adiciona um ponto na lista se a metrica eh maior do que o limite setado,
        points.append((int(x), int(y)) if conf > args.thr else None)
    ## AGora que temos todos os pontos, vamos definir o esqueleto que liga estes pontos            
    for pair in POSE_PAIRS:            
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)
        
        idFrom = BODY_PARTS[partFrom]  ## caracteriza o ponto de partida da linha
        idTo   = BODY_PARTS[partTo]    ## caracteriza o ponto de chegada da linha
       
        if points[idFrom] and points[idTo]:   ## se idFrom e idTo coincidem, é o mesmo ponto!
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    t, _ = net.getPerfProfile()
    freq = cv.getTickFrequency() / 1000
    cv.putText(frame, '%.2fms' % (t / freq), (10, 20), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    cv.imshow('OpenPose using OpenCV', frame)                 